In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
from dask.distributed import Client

c = Client()
c

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:56525,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:56532,Total threads: 2
Dashboard: http://127.0.0.1:56535/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:56530,


In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import pyplot as plt

from py_clone_detective.clone_counters import LazyCloneCounter
from py_clone_detective.utils import generate_random_cmap

In [ ]:
exp = LazyCloneCounter(
    exp_name="Marcm2a_E7F1",
    img_name_regex=r"a\dg\d\dp\d",
    pixel_size=0.275,
    tot_seg_ch="C0",
)

exp.add_images(
    C0="data/MARCM_experiment/images/C0/*.tif",
    C1="data/MARCM_experiment/images/C1/*.tif",
    C2="data/MARCM_experiment/images/C2/*.tif",
    C3="data/MARCM_experiment/images/C3/*.tif",
)

exp.add_segmentations(C0="data/MARCM_experiment/segmentations/C0_stardist_segs/*.tif")

In [ ]:
exp.make_measurements()

In [ ]:
exp.results_measurements = exp.results_measurements.eval(
    "total_intensity = mean_intensity * area"
)

In [ ]:
exp.add_clones_and_neighbouring_labels(
    thresholds=['int_img_ch == "C1" & seg_ch == "C0" & mean_intensity > 1000'],
    thresh_name="GFP",
    calc_clones=True,
)

In [ ]:
exp.add_clones_and_neighbouring_labels(
    thresholds=[
        'int_img_ch == "C2" & seg_ch == "C0" & total_intensity > 5e5',
        'int_img_ch == "C3" & seg_ch == "C0" & mean_intensity < 5000',
    ],
    thresh_name="EC",
    calc_clones=False,
)

In [ ]:
exp.add_clones_and_neighbouring_labels(
    thresholds=[
        'int_img_ch == "C2" & seg_ch == "C0" & total_intensity < 5e5',
        'int_img_ch == "C3" & seg_ch == "C0" & mean_intensity > 5000',
    ],
    thresh_name="EE",
    calc_clones=False,
)

In [ ]:
exp.add_clones_and_neighbouring_labels(
    thresholds=["not EC_pos & not EE_pos & area_um2 > 50"],
    thresh_name="preEC",
    calc_clones=False,
)

In [ ]:
exp.add_clones_and_neighbouring_labels(
    thresholds=["not EC_pos & not EE_pos & area_um2 < 50"],
    thresh_name="ISCorEB",
    calc_clones=False,
)

In [ ]:
exp.measure_clones_and_neighbouring_labels(thresh_name="GFP")
exp.measure_clones_and_neighbouring_labels(thresh_name="EC")
exp.measure_clones_and_neighbouring_labels(thresh_name="EE")
exp.measure_clones_and_neighbouring_labels(thresh_name="preEC")
exp.measure_clones_and_neighbouring_labels(thresh_name="ISCorEB")

In [ ]:
df = exp.combine_neighbour_counts_and_measurements().copy()

In [ ]:
df.to_csv('data/example_results.csv')

In [ ]:
def mutually_exclusive_cell_types(self):
    return (self.results_measurements.filter(regex='_pos').sum(axis=1) == 1).all()

In [ ]:
exp.results_measurements.query('EC_pos == EE_pos == True')

,seg_ch,int_img_ch,seg_img,int_img,label,area,mean_intensity,centroid-0,centroid-1,area_um2,total_intensity,GFP_clonepos,EC_pos,EE_pos,preEC_pos,ISCorEB_pos
276,C0,C0,a1g01p1,a1g01p1,70,633,5179.017378,246.238547,677.533965,47.870625,3278318.0,True,True,True,False,False
277,C0,C1,a1g01p1,a1g01p1,70,633,5321.210111,246.238547,677.533965,47.870625,3368326.0,True,True,True,False,False
278,C0,C2,a1g01p1,a1g01p1,70,633,1456.003160,246.238547,677.533965,47.870625,921650.0,True,True,True,False,False
279,C0,C3,a1g01p1,a1g01p1,70,633,5870.906793,246.238547,677.533965,47.870625,3716284.0,True,True,True,False,False
560,C0,C0,a1g01p1,a1g01p1,141,882,5170.695011,419.170068,119.404762,66.701250,4560553.0,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76699,C0,C3,a2g09p3,a2g09p3,65,368,6457.035326,201.076087,315.375000,27.830000,2376189.0,False,True,True,False,False
78696,C0,C0,a2g10p2,a2g10p2,41,654,4658.697248,277.941896,733.157492,49.458750,3046788.0,False,True,True,False,False
78697,C0,C1,a2g10p2,a2g10p2,41,654,87.758410,277.941896,733.157492,49.458750,57394.0,False,True,True,False,False
78698,C0,C2,a2g10p2,a2g10p2,41,654,1997.519878,277.941896,733.157492,49.458750,1306378.0,False,True,True,False,False


In [ ]:
mutually_exclusive_cell_types(exp)

False

In [ ]:
exp.results_measurements.filter(regex='_pos').sum()

EC_pos         30984
preEC_pos        756
ISCorEB_pos    51608
EE_pos          3780
dtype: int64

In [ ]:
exp.results_measurements.filter(regex='pos')